# Quantrack Python crash course 12.

This notebook is organized as follows :

### 1. Data-loading from Binance API
### 2. Moving average calculation
### 3. Setting up alert system using smtplib

In this notebook, we will built a prototype of alert system

## 1. Data loading from Binance API

In [49]:
import pandas as pd
import numpy as np

#datetime package
from datetime import datetime

#Client connection package
from binance.client import Client

In [50]:
#Connect to client Using API-Key
api_key = 'YOUR_KEY'
api_secret = 'YOUR_KEY'

#connect using binance client
client = Client(api_key, api_secret)

We are now connected to Binance API, we will extract BTC/USDT hourly data . Note that Binance API, like most APIs, has some limitations :

* You can't retrieve more than 500 data-points
* You can't do more than 1200 requests per minute

In [51]:
candles_ETC_1HOUR = client.get_klines(symbol='ETCUSDT', interval=Client.KLINE_INTERVAL_1HOUR)

In [52]:
candles_ETC_1HOUR[0]

[1591236000000,
 '6.95550000',
 '7.01320000',
 '6.94560000',
 '6.99790000',
 '84010.26000000',
 1591239599999,
 '586906.07636200',
 2028,
 '37203.94000000',
 '260004.13720900',
 '0']

At this stage we retrieved data for 1hour-BTC, we want to convert this list of lists to a dev-friendly pandas DataFrame. Columns of interest are :

* Date
* Open price
* High price
* Low price
* Close price

In [53]:
#let's convert it to a Pandas DataFrame :
df_ETCUSDT_1HOUR_temp  = pd.DataFrame(candles_ETC_1HOUR)
df_ETCUSDT_1HOUR_temp

,0,1,2,3,4,5,6,7,8,9,10,11
0,1591236000000,6.95550000,7.01320000,6.94560000,6.99790000,84010.26000000,1591239599999,586906.07636200,2028,37203.94000000,260004.13720900,0
1,1591239600000,7.00440000,7.04170000,6.99250000,7.01470000,69791.54000000,1591243199999,489799.56323800,2065,37514.95000000,263319.85965100,0
2,1591243200000,7.01680000,7.01910000,6.97350000,6.98980000,56246.75000000,1591246799999,393463.87057800,1541,30235.72000000,211490.45160500,0
3,1591246800000,6.98860000,6.99710000,6.94000000,6.95410000,47262.12000000,1591250399999,329072.25280800,1404,24033.15000000,167318.43799800,0
4,1591250400000,6.95220000,6.97860000,6.91730000,6.95610000,46085.20000000,1591253999999,320242.75742500,1456,24258.48000000,168589.89496500,0
...,...,...,...,...,...,...,...,...,...,...,...,...
495,1593018000000,6.20480000,6.21870000,6.19280000,6.21870000,15983.86000000,1593021599999,99174.50889400,534,8193.31000000,50836.22891800,0
496,1593021600000,6.21890000,6.22250000,6.20190000,6.20720000,22203.39000000,1593025199999,137993.89602000,554,8721.99000000,54206.25578300,0
497,1593025200000,6.20810000,6.23010000,6.19180000,6.19970000,16897.22000000,1593028799999,104938.35634600,437,7006.33000000,43537.78193600,0
498,1593028800000,6.19970000,6.21660000,6.19230000,6.20420000,14808.53000000,1593032399999,91930.56557900,378,6248.44000000,38779.57980700,0


We would like to filter out columns of interests, rename them, verify types of objects, and make datetime column readable.

Note that we name them using capital letters as it is necessary for mplfinance plot later.

In [54]:
#let's create a preprocessing function :

def clean_data(df, time_index = True):
    
    #filter firstfour columns
    df = df.iloc[:,:5]
    
    #rename columns 
    df.columns = [
        'date',
        'open',
        'high',
        'low',
        'close'
    ]
    
    #refactor date column
    df['date'] = df['date'].apply(lambda x: datetime.fromtimestamp(x / 1000))
    
    #set date as index if index option is True, note that its True by default
    if time_index == True:
        df.index = df.date
        df = df.drop(['date'],axis=1)
        return df
    
    else:
        return df
        
    

* Note : time_index = True means index of our cleaned dataframe will be timespamps instead of range(1,len(df)) . time_index variable is set as True by default in the function, which means we do not need to specify if we want it to be True. 


In [74]:
#let's apply our pre-processing functions
df_ETCUSDT_1HOUR = clean_data(df_ETCUSDT_1HOUR_temp, time_index=True)

In [76]:
df_ETCUSDT_1HOUR

,open,high,low,close
date,,,,
2020-06-03 21:00:00,6.95550000,7.01320000,6.94560000,6.99790000
2020-06-03 22:00:00,7.00440000,7.04170000,6.99250000,7.01470000
2020-06-03 23:00:00,7.01680000,7.01910000,6.97350000,6.98980000
2020-06-04 00:00:00,6.98860000,6.99710000,6.94000000,6.95410000
2020-06-04 01:00:00,6.95220000,6.97860000,6.91730000,6.95610000
...,...,...,...,...
2020-06-24 12:00:00,6.20480000,6.21870000,6.19280000,6.21870000
2020-06-24 13:00:00,6.21890000,6.22250000,6.20190000,6.20720000
2020-06-24 14:00:00,6.20810000,6.23010000,6.19180000,6.19970000


Now that we have our data-set ready, we can start exploring basic *candlestick* visualization on Python 

##### Candlesticks, in four points :  

* Candlestick charts are used by traders to determine possible price movement based on past patterns.
* Candlesticks are useful when trading as they show four price points (open, close, high, and low) throughout the period of time the trader specifies.
* Many algorithms are based on the same price information shown in candlestick charts.
* Trading is often dictated by emotion, which can be read in candlestick charts.

## 2. Moving average calculation

We would like to calculate 5 and 10 moving average values and check when they are crossing each other.

In [57]:
df_ETCUSDT_1HOUR = df_ETCUSDT_1HOUR.astype(float)

#### Let's define non-generic function

In [58]:
def sma_5(df):
    return df['close'].rolling(5, min_periods=0).mean()

def sma_10(df):
    return df['close'].rolling(10, min_periods=0).mean()

#### Let's add the two moving average

In [77]:
df_ETCUSDT_1HOUR['sma_5'] = sma_5(df_ETCUSDT_1HOUR)
df_ETCUSDT_1HOUR['sma_10'] = sma_10(df_ETCUSDT_1HOUR)

#let's trick the index a little bit
df_ETCUSDT_1HOUR['date'] = df_ETCUSDT_1HOUR.index
df_ETCUSDT_1HOUR.index = range(len(df_ETCUSDT_1HOUR))

In [79]:
df_ETCUSDT_1HOUR

,open,high,low,close,sma_5,sma_10,date
0,6.95550000,7.01320000,6.94560000,6.99790000,6.997900,6.997900,2020-06-03 21:00:00
1,7.00440000,7.04170000,6.99250000,7.01470000,7.006300,7.006300,2020-06-03 22:00:00
2,7.01680000,7.01910000,6.97350000,6.98980000,7.000800,7.000800,2020-06-03 23:00:00
3,6.98860000,6.99710000,6.94000000,6.95410000,6.989125,6.989125,2020-06-04 00:00:00
4,6.95220000,6.97860000,6.91730000,6.95610000,6.982520,6.982520,2020-06-04 01:00:00
...,...,...,...,...,...,...,...
495,6.20480000,6.21870000,6.19280000,6.21870000,6.216460,6.264160,2020-06-24 12:00:00
496,6.21890000,6.22250000,6.20190000,6.20720000,6.200640,6.249860,2020-06-24 13:00:00
497,6.20810000,6.23010000,6.19180000,6.19970000,6.197700,6.233710,2020-06-24 14:00:00
498,6.19970000,6.21660000,6.19230000,6.20420000,6.206840,6.226740,2020-06-24 15:00:00


Let's naively check previous crossovers.

In [70]:
previous_5 = df_ETCUSDT_1HOUR['sma_5'].shift(1)
previous_10 = df_ETCUSDT_1HOUR['sma_10'].shift(1)

crossing = (((df_ETCUSDT_1HOUR['sma_5'] <= df_ETCUSDT_1HOUR['sma_10']) & (previous_5 >= previous_10))
            | ((df_ETCUSDT_1HOUR['sma_5'] >= df_ETCUSDT_1HOUR['sma_10']) & (previous_5 <= previous_10)))

crossing_dates = df_ETCUSDT_1HOUR.loc[crossing, 'date']

print(crossing_dates)

1     2020-06-03 22:00:00
2     2020-06-03 23:00:00
3     2020-06-04 00:00:00
4     2020-06-04 01:00:00
5     2020-06-04 02:00:00
              ...        
455   2020-06-22 20:00:00
468   2020-06-23 09:00:00
476   2020-06-23 17:00:00
481   2020-06-23 22:00:00
487   2020-06-24 04:00:00
Name: date, Length: 69, dtype: datetime64[ns]


## 3. Sending e-mail

In [102]:
import smtplib, ssl

In [ ]:
import smtplib, ssl

port = 465  # For SSL
password = input("Enter your password here")

# Create a secure SSL context
context = ssl.create_default_context()

with smtplib.SMTP_SSL("smtp.gmail.com", port, context=context) as server:
    server.login("kerlon.mcdouglas@gmail.com", password)

In [ ]:
crossing_dates

In [ ]:
list_of_dates = list(crossing_dates)

In [113]:
import smtplib

d = list_of_dates[-1]

gmail_user = 'kerlon.mcdouglas@gmail.com'
gmail_password = 'YOURPASSWORD'

sent_from = gmail_user
to = ['EXAMPLE@example.com']
subject = 'OMG Super Important Message'
body = 'Las medias moviles se cruceron por la ultima vez en la fecha {}'.format(d)

email_text = """\
From: %s
To: %s
Subject: %s

%s
""" % (sent_from, ", ".join(to), subject, body)

try:
    server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    server.ehlo()
    server.login(gmail_user, gmail_password)
    server.sendmail(sent_from, to, email_text)
    server.close()

    print ('Email sent!')
except:
    print ('Something went wrong...')

Email sent!
